# STEP 1: Import Required Libraries

In [42]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity


# STEP 2: Load Dataset

In [43]:
df = pd.read_csv("Dataset .csv")
df.head()

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229


# STEP 3: Understand the Dataset

In [44]:
df.columns

Index(['Restaurant ID', 'Restaurant Name', 'Country Code', 'City', 'Address',
       'Locality', 'Locality Verbose', 'Longitude', 'Latitude', 'Cuisines',
       'Average Cost for two', 'Currency', 'Has Table booking',
       'Has Online delivery', 'Is delivering now', 'Switch to order menu',
       'Price range', 'Aggregate rating', 'Rating color', 'Rating text',
       'Votes'],
      dtype='object')

In [45]:
# Checking Null values

df.isnull().sum()

Restaurant ID           0
Restaurant Name         0
Country Code            0
City                    0
Address                 0
Locality                0
Locality Verbose        0
Longitude               0
Latitude                0
Cuisines                9
Average Cost for two    0
Currency                0
Has Table booking       0
Has Online delivery     0
Is delivering now       0
Switch to order menu    0
Price range             0
Aggregate rating        0
Rating color            0
Rating text             0
Votes                   0
dtype: int64

In [46]:
#Convert Column into Lower Case

df.columns = df.columns.str.strip().str.lower() 
df.columns

Index(['restaurant id', 'restaurant name', 'country code', 'city', 'address',
       'locality', 'locality verbose', 'longitude', 'latitude', 'cuisines',
       'average cost for two', 'currency', 'has table booking',
       'has online delivery', 'is delivering now', 'switch to order menu',
       'price range', 'aggregate rating', 'rating color', 'rating text',
       'votes'],
      dtype='object')

# STEP 4: Data Preprocessing (Handling Missing Values)

In [47]:
df['cuisines'] = df['cuisines'].fillna('Unknown')
df['price range'] = df['price range'].fillna(df['price range'].mode()[0])
df['aggregate rating'] = df['aggregate rating'].fillna(df['aggregate rating'].mean())


# STEP 5: ENCODE CATEGORICAL COLUMNS 

In [48]:

le_cuisine = LabelEncoder()
le_price = LabelEncoder()

df['cuisine encoded'] = le_cuisine.fit_transform(df['cuisines'])
df['price encoded'] = le_price.fit_transform(df['price range'])


# STEP 6: CREATE FEATURE MATRIX

In [53]:
features = df[['cuisine encoded', 'price encoded', 'aggregate rating']]


# STEP 7: DEFINE RECOMMENDATION FUNCTION

In [54]:
def recommend_restaurants(user_cuisine, user_price, top_n=5):

    # Encode user preferences
    cuisine_val = le_cuisine.transform([user_cuisine])[0]
    price_val = le_price.transform([user_price])[0]

    # Create user preference vector
    user_vector = np.array([
        [cuisine_val, price_val, df['aggregate rating'].mean()]
    ])

    # Calculate similarity scores
    similarity_scores = cosine_similarity(user_vector, features)[0]

    # Get top N restaurants
    top_indices = similarity_scores.argsort()[-top_n:][::-1]

    # Return recommendations
    return df.iloc[top_indices][
        ['restaurant name', 'cuisines', 'price range', 'aggregate rating']
    ]


# STEP 8: TEST

In [55]:
recommend_restaurants(
    user_cuisine="Italian",
    user_price=2,   # must match dataset
    top_n=3
)


,restaurant name,cuisines,price range,aggregate rating
7753,Slice of Italy,"Italian, Pizza, Bakery",2,2.8
5148,Pizza Hut Delivery,"Italian, Pizza, Fast Food",2,2.8
6036,Pizza Hut Delivery,"Italian, Pizza, Fast Food",2,2.8


In [56]:
print("Recommended restaurants match the user's cuisine preference and price range.")


Recommended restaurants match the user's cuisine preference and price range.
